In [ ]:
import os, torch, torch.nn as nn, torchvision, matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from torchvision.io import read_image
import PIL.Image as Image
import re

img_folder = '/Users/liam/Documents/Python/working/oxford-iiit-pet/images/'
# ann_folder = '/Users/liam/Documents/Python/working/oxford-iiit-pet/annotations/annotations/trimaps'

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(device)

In [ ]:
# remove all files that are not images (jpg)
for filename in os.listdir(img_folder):
    if not filename.endswith('.jpg'):
        os.remove(img_folder+filename)

In [ ]:

filenames=[]
for filename in os.listdir(img_folder):
    filenames.append(filename)
    
# get the labels from the image filenames via regular expressions
# format: <class>_<color>_<number>.jpg
# e.g. british_shorthair_100.jpg

labels=[]
for filename in filenames:
    label = re.split('_', filename)
    if len(label) == 2:
        label = label[0]
    else:
        label = label[0]+"_"+label[1]
    labels.append(label)

# print unique labels
print(set(labels))

# count classes in labels
from collections import Counter
Counter(labels)

In [ ]:
classes = list(set(labels))
classes_idx = {classes[i]:i for i in range(len(classes))}

In [ ]:
class OxfordDataset(Dataset):
    def __init__(self, filenames, labels, transform=None):
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        try:
            image = Image.open(img_folder+self.filenames[idx])
            image = image.convert('RGB')
            label = classes_idx[self.labels[idx]]
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            print(e)
            print("Error loading image: ", self.filenames[idx], "idx", idx)

transform=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224), antialias=True),
    torchvision.transforms.RandomHorizontalFlip(0.5),
    torchvision.transforms.ColorJitter(brightness=0.05, contrast=0.1, saturation=0.05, hue=0.05),
    # random erasing
    torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value=0, inplace=False),
    torchvision.transforms.RandomRotation(degrees=10),
])

sample_dataset=OxfordDataset(filenames[:20], labels[:20], transform=transform)
sample_dataloader=DataLoader(sample_dataset, batch_size=10, shuffle=True)

# visualize a batch of images using the dataloader
fig, ax = plt.subplots(2, 5, figsize=(10, 5))
index=0
for xb, yb in sample_dataloader:
    for row in range(2):
        for col in range(5):
            ax[row, col].imshow(xb[row*5+col].permute(1, 2, 0))
            ax[row, col].set_title(classes[yb[row*5+col].item()])
            ax[row, col].axis('off')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(filenames, labels, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
class OxfordCNN(nn.Module):
    def __init__(self, p=0.2):
        super(OxfordCNN, self).__init__()
        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(32),
            torch.nn.Dropout(p)
        ) # 32x112x112
        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32, 128, kernel_size=3, stride=2, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(128),
            torch.nn.Dropout(p)
        ) # 128x28x28
        self.layer3=torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(256),
            torch.nn.Dropout(p)
        ) # 256x7x7
        self.layer4=torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.ReLU(),
            torch.nn.BatchNorm2d(512),
            torch.nn.Dropout(p)
        ) # 512x2x2
        self.fc1=torch.nn.Sequential(
            torch.nn.Linear(512*2*2, 1000),
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(1000)
        ) # 1000
        self.fc2=torch.nn.Sequential(
            torch.nn.Linear(1000, 2),
            torch.nn.Softmax(dim=1)
        ) # 2

    def forward(self, xb):
        out=self.layer1(xb) # 32x224x224
        out=self.layer2(out) # 128x56x56
        out=self.layer3(out) # 256x14x14
        out=self.layer4(out) # 512x7x7
        out=out.reshape(out.size(0), -1) # 512x2x2 -> 2048
        out=self.fc1(out) # 1000
        out=self.fc2(out) # 2
        return out
    
model=OxfordCNN()
model(xb).shape

In [ ]:
def get_accuracy(model, loader):
    correct=0
    #print('xb: '+type(xb)+', yb: '+type(yb)) -> throws reference error
    for xb, yb in loader:
        xb=xb.to(device)
        yb=yb.to(device)
        y_hat=model(xb)
        y_pred=torch.argmax(y_hat, dim=1)
        correct+=(y_pred==yb).sum().item()
    return correct/len(loader)

In [ ]:
'''
# convert yb to one-hot encoding
def one_hot(yb, num_classes):
    yb_onehot=torch.zeros(yb.shape[0], num_classes)
    yb_onehot[torch.arange(yb.shape[0]), yb]=1
    return yb_onehot
'''
batch_size=128

train_dataset=OxfordDataset(X_train, y_train, transform=transform)
train_dataloader=DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset=OxfordDataset(X_test, y_test, transform=transform)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# hyperparameters
p=0.2
EPOCHS=10
batch_size=10
lr=0.01

loss_fn=torch.nn.CrossEntropyLoss()

optimizer=torch.optim.Adam(model.parameters(), lr=lr)

def train_batch(model, optimizer, xb, yb):
    xb=xb.to(device)
    yb=yb.to(device)
    y_hat=model(xb)
    loss=loss_fn(y_hat, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()

def train_epoch(model, optimizer,  x, y):
    losses=[]
    for xb, yb in train_dataloader:
        losses.append(train_batch(model, optimizer, xb, yb))
    return losses, get_accuracy(model, test_dataloader)


In [ ]:
model = OxfordCNN().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=lr)

accs=[]
# define tqdm bar
bar=tqdm(range(EPOCHS))
for i in bar:
    losses, acc=train_epoch(model, optimizer, X_train, y_train)
    accs.append(acc)
    bar.set_description(f"Accuracy: {acc:.3f}")
plt.plot(accs) #